# **Step1_AI면접관 Agent v1.0**

## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 라이브러리

In [9]:
!pip install -r /content/drive/MyDrive/AIVLE/02_ai_mini_project/project_genai/requirements.txt -q

In [10]:
!pip install -U langchain

  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
Using cached langchain-0.3.25-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.24
    Uninstalling langchain-0.3.24:
      Successfully uninstalled langchain-0.3.24


### (3) OpenAI API Key 확인
* api_key.txt 파일에 다음의 키를 등록하세요.
    * OPENAI_API_KEY
    * NGROK_AUTHTOKEN

In [11]:
import os

def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/AIVLE/02_ai_mini_project/project_genai/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

In [12]:
print(os.environ['OPENAI_API_KEY'][:30])

sk-proj-bPUAGD4uXbVR0ETvXjPmul


## **2. App.py**

* 아래 코드에, Step1 혹은 고도화 된 Step2 파일 코드를 붙인다.
    * 라이브러리
    * 함수들과 그래프
* Gradio 코드는 그대로 사용하거나 일부 수정 가능

In [13]:
%%writefile app.py

####### 여러분의 함수와 클래스를 모두 여기에 붙여 넣읍시다. #######

## 1. 라이브러리 로딩 ---------------------------------------------
import os
import ast
import fitz  # PyMuPDF
import random
import openai
import warnings
import pandas as pd
import numpy as np
from typing import List, Dict, TypedDict

from docx import Document

from langchain import hub
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph, START, END
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_community.document_loaders import CSVLoader
from typing import Literal
from langchain_core.output_parsers import JsonOutputParser

warnings.filterwarnings("ignore", category=DeprecationWarning)


## ---------------- 1단계 : 사전준비 ----------------------

# 1) 파일 입력 --------------------
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text

    elif ext == ".docx":
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())

    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")


# 2) State 선언 --------------------
class InterviewState(TypedDict):
    # 고정 정보
    resume_text: str
    resume_summary: str
    resume_keywords: List[str]
    question_strategy: Dict[str, Dict]

    # 인터뷰 로그
    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation: List[Dict[str, str]]
    next_step: str

    # 추가
    covered_strategies: List[str]
    reflection_status: str


# 파일에서 읽어 State 초기화
path = '/content/drive/MyDrive/AIVLE/02_ai_mini_project/project_genai/'
file_path = path + 'Resume_sample.pdf'

# 텍스트 추출
resume_text = extract_text_from_file(file_path)

# 초기 상태 구성
initial_state: InterviewState = {
    "resume_text": resume_text,
    "resume_summary": '',
    "resume_keywords": [],
    "question_strategy": {},

    "current_question": '',
    "current_answer": '',
    "current_strategy": '',
    "conversation": [],
    "evaluation": [],
    "next_step": '',
    "covered_strategies": [],
    "reflection_status": '',
}


# 3) resume 분석 --------------------
def analyze_resume(state: InterviewState) -> InterviewState:
    resume_text = state["resume_text"]

    # 출력 파서: 요약은 str, 키워드는 쉼표 리스트
    parser1 = StrOutputParser()
    parser2 = CommaSeparatedListOutputParser()

    # 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "너는 인사담당자이자 면접관이다. 지금부터 지원자의 이력서와 자기소개서를 바탕으로 "
            "면접 준비를 고도화하고, 핵심 내용을 요약하며, 맞춤형 질문 전략 수립을 위한 기초 자료를 도출한다. "
            "텍스트에서 지원자의 핵심 경력, 직무역량, 주요 성과, 성장경로, 강점, 약점, 특이사항, 면접에서 주목해야 할 포인트를 분석하라. "
            "이 요약과 키워드는 이후 개인화된 질문과 평가 전략 수립에 반드시 활용된다."
        ),
        (
            "human",
            "질문:\n{query}\n이력서 및 자기소개서 텍스트:\n{context}"
        ),
        (
            "system",
            "{format_instructions}"
        )
    ])

    # 메시지 구성: 요약 요청
    messages1 = prompt.format_messages(
        query=(
            "이력서와 자기소개서의 핵심 내용을 3~5문장으로 요약하라. "
            "핵심 경력, 직무역량, 주요 성과, 성장경로, 강점, 약점, 특이사항, 주목 포인트가 빠지지 않도록 정리하라."
        ),
        context=resume_text,
        format_instructions="간결하고 명확하게 한글로 요약."
    )

    # 메시지 구성: 키워드 요청
    messages2 = prompt.format_messages(
        query="이력서와 자기소개서에서 맞춤형 질문 전략 수립에 반드시 참고해야 할 주요 키워드 10개를 추출하라.",
        context=resume_text,
        format_instructions="쉼표로 구분하여 10개의 한글 키워드만 추출."
    )

    # GPT-4o 모델 설정
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

    # 응답 생성
    response1 = llm.invoke(messages1)
    response2 = llm.invoke(messages2)

    # 응답 파싱
    resume_summary = parser1.parse(response1.content)
    resume_keywords = parser2.parse(response2.content)

    # 상태 반환
    return {
        **state,
        "resume_summary": resume_summary,
        "resume_keywords": resume_keywords,
    }


# 4) 질문 전략 수립 --------------------

def generate_question_strategy(state: InterviewState) -> InterviewState:
    resume_summary = state['resume_summary']
    resume_keywords = state['resume_keywords']

    # LLM 설정
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

    # 출력 파서 설정
    parser = JsonOutputParser()

    # 프롬프트 구성
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "너는 AI 기반 인사담당자이자 면접관이며, 사전준비고도화, 이력서 요약, 전략 수립의 전문가이다. "
            "지원자의 이력서와 자기소개서 분석 결과를 바탕으로, 맞춤형 면접 질문 전략을 수립하는 역할을 맡고 있다. "
            "각 전략은 지원자의 실제 경력, 역량, 동기, 커뮤니케이션, 논리적 사고를 평가하는 데 초점을 맞춘다."
        ),
        (
            "human",
            """
            아래는 지원자에 대한 핵심 요약과 주요 키워드이다.

            [지원자 핵심 요약]
            {summary}

            [지원자 주요 키워드]
            {keywords}

            위 정보를 바탕으로, 다음 3가지 분야별로 질문 전략을 수립하라.

            1. 경력 및 경험
            - 지원자의 실무 경험, 프로젝트, 전공, 주요 성과 등 실제 업무와 관련된 배경을 평가할 수 있도록 질문 방향을 설정하라.

            2. 동기 및 커뮤니케이션
            - 지원자의 지원 동기, 협업 경험, 조직 적응력, 대인관계 역량 등을 파악할 수 있도록 질문 방향을 설정하라.

            3. 논리적 사고
            - 지원자의 문제 해결력, 사고 과정, 의사 결정 방식 등을 평가할 수 있도록 질문 방향을 설정하라.

            각 분야별로 아래 두 정보를 반드시 작성하라:
            - "방향": 이 분야에서 질문하고자 하는 목적과 방향성을 1문장으로 요약
            - "예시질문": 실제 면접관이 사용할 수 있도록, 지원자 정보에 맞춘 구체적인 질문 2개

            **출력 형식은 반드시 다음 JSON 구조를 따르며, 필드명은 그대로 유지한다:**
            {{
              "경력 및 경험": {{
                "방향": "...",
                "예시질문": ["...", "..."]
              }},
              "동기 및 커뮤니케이션": {{
                "방향": "...",
                "예시질문": ["...", "..."]
              }},
              "논리적 사고": {{
                "방향": "...",
                "예시질문": ["...", "..."]
              }}
            }}

            작성하는 질문은 반드시 지원자의 요약 및 키워드 내용을 반영하여, 실제 지원자 맞춤형 질문이 되도록 한다.
            {format_instructions}
            """
        )
    ])

    # 체인 구성 및 실행
    chain = prompt | llm | parser

    strategy_dict = chain.invoke({
        "summary": resume_summary,
        "keywords": ", ".join(resume_keywords),
        "format_instructions": parser.get_format_instructions()
    })

    return {
        **state,
        "question_strategy": strategy_dict
    }


# 5) 1단계 하나로 묶기 --------------------

def preProcessing_Interview(file_path: str) -> InterviewState:
    # 1. 이력서 텍스트 추출
    resume_text = extract_text_from_file(file_path)

    # 2. 초기 상태 구성
    state: InterviewState = {
        "resume_text": resume_text,
        "resume_summary": "",
        "resume_keywords": [],
        "question_strategy": {},
        "current_question": "",
        "current_answer": "",
        "current_strategy": "",
        "conversation": [],
        "evaluation": [],
        "next_step": "",
        "covered_strategies": [],
        "reflection_status": ""
    }

    # 3. 이력서 요약 및 키워드 분석
    state = analyze_resume(state)

    # 4. 질문 전략 수립
    state = generate_question_strategy(state)

    # 5. 첫 번째 질문 구성: '경력 및 경험'의 예시 질문 중 첫 번째 선택
    question_strategy = state.get("question_strategy", {})
    example_questions = question_strategy.get("경력 및 경험", {}).get("예시질문", [])

    selected_question = example_questions[0] if example_questions else ""

    return {
        **state,
        "current_question": selected_question,
        "current_strategy": "경력 및 경험"
    }



## ---------------- 2단계 : 면접 Agent ----------------------

# 1) 답변 입력 --------------------
def update_current_answer(state: InterviewState, user_answer: str) -> InterviewState:
    return {
        **state,
        "current_answer": user_answer.strip()
    }

# 2) 답변 평가 --------------------
def re_evaluation_answer(state: InterviewState) -> InterviewState:
    question = state["current_question"]
    answer = state["current_answer"]

    response_schemas = [
        ResponseSchema(
            name="기술 이해도",
            description=(
                "면접자가 언급한 기술의 개념이나 원리를 정확히 이해하고 설명하고 있는지 평가한다.\n"
                "- 상: 개념과 작동 원리를 정확히 설명하며 실용적인 맥락까지 언급함.\n"
                "- 중: 개념은 언급하지만 부정확하거나 얕은 수준의 설명.\n"
                "- 하: 기술을 오용하거나 틀린 정의를 내림."
            )
        ),
        ResponseSchema(
            name="경험의 구체성",
            description=(
                "해당 기술을 실제로 사용한 경험을 구체적으로 설명했는지 평가한다.\n"
                "- 상: 프로젝트, 역할, 기간 등 맥락이 명확하고 세부 설명이 풍부함.\n"
                "- 중: 경험 언급은 있으나 구체적인 사례나 역할이 불명확함.\n"
                "- 하: 단순히 '써봤다'고 말하는 수준, 경험이 모호하거나 없음."
            )
        ),
        ResponseSchema(
            name="문제 해결 능력",
            description=(
                "기술을 통해 어떤 문제를 해결했는지를 평가한다.\n"
                "- 상: 문제 해결 과정과 기술의 기여도, 결과를 구체적으로 제시함.\n"
                "- 중: 해결 사례 언급은 있으나 연결고리가 약하거나 결과가 모호함.\n"
                "- 하: 문제 해결과의 관련성을 확인할 수 없음."
            )
        )
    ]

    parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = parser.get_format_instructions()

    prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 AI 기반 면접 평가 시스템이다. 다음 질문과 면접자의 답변을 바탕으로 면접자의 기술 역량을 평가한다."),
        ("human", """
[질문]
{question}

[답변]
{answer}

다음 세 항목에 대해 각각 평가해라. 각 항목에 대해 반드시 아래의 **상·중·하** 기준 중 하나만 선택할 것:

1. 기술 이해도
- 개념과 작동 원리를 정확히 설명하며 실용적인 맥락까지 언급 → 상
- 개념은 언급하지만 부정확하거나 얕은 수준의 설명 → 중
- 기술을 오용하거나 틀린 정의를 내림 → 하

2. 경험의 구체성
- 프로젝트, 역할, 기간 등 맥락이 명확하고 세부 설명이 풍부 → 상
- 경험 언급은 있으나 구체적인 사례나 역할이 불명확 → 중
- 단순히 '써봤다'는 수준, 경험이 모호하거나 없음 → 하

3. 문제 해결 능력
- 문제 해결 과정과 기술의 기여도, 결과를 구체적으로 제시함 → 상
- 해결 사례 언급은 있으나 연결고리가 약하거나 결과가 모호함 → 중
- 문제 해결과의 관련성을 확인할 수 없음 → 하

{format_instructions}
""")
    ])

    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    messages = prompt.format_messages(
        question=question,
        answer=answer,
        format_instructions=format_instructions
    )

    response = llm.invoke(messages)
    evaluation_result = parser.parse(response.content)

    evaluation = state.get("evaluation", [])
    evaluation.append({**evaluation_result})
    state["evaluation"] = evaluation
    state["next_step"] = "decide_next_step"

    return state

# 평가 결과 적절성 되돌아보기
def reflection_node(state: InterviewState) -> InterviewState:
    latest_eval = state["evaluation"][-1]
    comment = latest_eval.get("comment", "")

    if "불명확" in comment or "모호" in comment:
        state["reflection_status"] = "재평가 필요"
        state["next_step"] = "re_evaluate_answer"
    else:
        state["reflection_status"] = "정상"
        state["next_step"] = "decide_next_step"

    conversation_exists = any(
        conv["question"] == state["current_question"] and conv["answer"] == state["current_answer"]
        for conv in state.get("conversation", [])
    )

    if not conversation_exists:
        state["conversation"].append({
            "question": state["current_question"],
            "answer": state["current_answer"],
            "strategy": state.get("current_strategy", "")
        })

    return state

# 평가 항목 추가: 질문과의 연관성 / 구체성
def evaluate_answer(state: InterviewState) -> InterviewState:
    question = state["current_question"]
    answer = state["current_answer"]

    response_schemas = [
        ResponseSchema(
            name="질문과의 연관성",
            description="답변이 질문에 적절하게 대응하는지 평가한다. (상: 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸 / 중: 질문과 관련은 있지만 핵심 포인트가 일부 누락됨 / 하: 질문과 관련이 약하거나 엉뚱한 내용 중심)"
        ),
        ResponseSchema(
            name="답변의 구체성",
            description="답변이 구체적인 사례나 근거를 포함하는지 평가한다. (상: 구체적인 예시 포함 / 중: 일반적인 설명 / 하: 모호하고 추상적임)"
        )
    ]

    parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = parser.get_format_instructions()

    prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 AI 기반 면접 평가 시스템이다. 다음 질문과 면접자의 답변을 바탕으로 지정된 평가 항목에 대해 객관적으로 평가한다."),
        ("human", """
[질문]
{question}

[답변]
{answer}

다음 두 항목에 대해 각각 평가해라:

1. 질문과의 연관성
- 답변이 질문의 핵심 의도에 적절히 대응하는가?
- (상: 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸 / 중: 질문과 관련은 있지만 핵심 포인트가 일부 누락됨 / 하: 질문과 관련이 약하거나 엉뚱한 내용 중심)

2. 답변의 구체성
- 답변이 구체적인 사례나 근거를 포함하고 있는가?
- (상: 구체적인 예시와 근거 포함 / 중: 일반적 설명 수준 / 하: 모호하고 추상적임)

각 항목에 대해 **등급(상, 중, 하)**만 반환할것. 그 외 표현(예: 좋음, 보통 등)은 절대 사용하지 말것.

{format_instructions}
""")
    ])

    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    messages = prompt.format_messages(
        question=question,
        answer=answer,
        format_instructions=format_instructions
    )

    response = llm.invoke(messages)
    evaluation_result = parser.parse(response.content)

    state["evaluation"].append({**evaluation_result})
    state["conversation"].append({
        "question": question,
        "answer": answer,
        "strategy": state.get("current_strategy", "")
    })

    current_strategy = state.get("current_strategy", "")
    if current_strategy:
        covered = set(state.get("covered_strategies", []))
        covered.add(current_strategy)
        state["covered_strategies"] = list(covered)

    return state

# 3) 인터뷰 진행 여부 결정 --------------------
def decide_next_step(state: InterviewState) -> InterviewState:
    conversation = state.get("conversation", [])
    evaluation = state.get("evaluation", [])
    num_turns = len(conversation)

    target_strategies = {"경력 및 경험", "동기 및 커뮤니케이션", "논리적 사고"}
    covered_strategies = {item.get("strategy") for item in conversation if "strategy" in item}
    has_covered_all_strategies = covered_strategies >= target_strategies

    if has_covered_all_strategies or num_turns >= 5:
        next_step = "end"
    elif evaluation:
        latest_score = evaluation[-1].get("score", "").strip()
        if latest_score == "하":
            next_step = "additional_question"
        elif latest_score in {"중", "상"}:
            next_step = "next_strategy"
        else:
            next_step = "additional_question"
    else:
        next_step = "additional_question"

    return {
        **state,
        "next_step": next_step
    }
# -----------------------------------------------
# 1. CSV 질문 예시 불러오기 및 벡터DB 구축
# -----------------------------------------------
# CSV 파일 로드
csv_path = path + "분야별_질문_예시.csv"
csv_loader = CSVLoader(file_path=csv_path)
documents_csv = csv_loader.load()

# 벡터 DB 정의
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(documents_csv, embedding,
                                    persist_directory="chroma_db")

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})


# -----------------------------------------------
# 2. 질문 생성 함수
# -----------------------------------------------
def generate_question(state: InterviewState) -> InterviewState:
    resume_summary = state.get("resume_summary", "")
    resume_keywords = ", ".join(state.get("resume_keywords", []))
    question_strategy = state.get("question_strategy", {})
    current_strategy = state.get("current_strategy", "")
    strategy = question_strategy.get(current_strategy, {}).get("방향", "")
    current_question = state.get("current_question", "")
    current_answer = state.get("current_answer", "")
    evaluation = state.get("evaluation", [])
    last_evaluation = evaluation[-1]

    search_query = f"{current_strategy} 관련 질문, 키워드: {resume_keywords}, 이전 질문: {current_question}"
    retrieved_docs = retriever.invoke(search_query)
    reference_questions = [doc.page_content.strip() for doc in retrieved_docs[:3]]
    reference_text = "\n".join([f"- {q}" for q in reference_questions]) if reference_questions else "- 없음"

    prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 AI 기반 면접관이다. 지금부터 지원자의 답변을 분석하고, 더 깊이 있는 사고를 유도할 수 있는 심화 질문을 설계해야 한다."),
        ("human",
         "지원자 이력서 요약:\n{resume_summary}\n\n"
         "핵심 키워드:\n{resume_keywords}\n\n"
         "질문 전략({current_strategy}):\n{strategy}\n\n"
         "이전 질문: {current_question}\n"
         "이전 답변: {current_answer}\n"
         "이전 답변에 대한 평가: {last_evaluation}\n\n"
         "다음은 유사한 질문 예시입니다. 이를 참고하여 새로운 질문을 만드세요:\n"
         "{reference_questions}\n\n"
         "**지원자의 생각을 더 확장시킬 수 있는 심화 질문**을 하나 생성하세요.\n"
         "- 질문은 이전 질문과 논리적으로 연결되어야 합니다.\n"
         "- 특히, 답변에서 부족했던 부분이나 추가로 설명해볼 만한 부분을 더 깊이 탐색하세요.\n"
         "- 면접관이 바로 사용할 수 있도록 **간결하고 구체적인 한 문장**으로 작성하세요."
        )
    ])

    messages = prompt.format_messages(
        resume_summary=resume_summary,
        resume_keywords=resume_keywords,
        current_strategy=current_strategy,
        strategy=strategy,
        current_question=current_question,
        current_answer=current_answer,
        last_evaluation=str(last_evaluation),
        reference_questions=reference_text
    )

    llm = ChatOpenAI(model="gpt-4o-mini")
    response = llm.invoke(messages)

    return {
        **state,
        "current_question": response.content.strip(),
        "current_answer": ""
    }


# -----------------------------------------------
# 3. 인터뷰 피드백 보고서 생성
# -----------------------------------------------
def summarize_interview(state: InterviewState) -> InterviewState:
    conversation = state.get("conversation", [])
    evaluation = state.get("evaluation", [])
    strategy = state.get("question_strategy", {})
    sections = []

    for idx, (qa, ev) in enumerate(zip(conversation, evaluation)):
        question = qa["question"]
        answer = qa["answer"]
        eval_relevance = ev.get("질문과의 연관성", "")
        eval_specificity = ev.get("답변의 구체성", "")

        prompt = ChatPromptTemplate.from_messages([
            ("system", "너는 면접 피드백 전문가이다. 아래 질문, 답변, 평가 결과를 참고하여 답변 요약, 강점, 약점을 간결히 작성하라."),
            ("human", f"""
[질문]
{question}

[답변]
{answer}

[평가]
- 질문과의 연관성: {eval_relevance}
- 답변의 구체성: {eval_specificity}

각 항목은 다음 형식으로 출력하라:

- 답변 요약: ...
- 강점: ...
- 약점: ...
""")
        ])

        messages = prompt.format_messages()
        llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
        llm_response = llm.invoke(messages)
        response = StrOutputParser().parse(llm_response.content)

        strategy_type = qa.get("strategy", f"질문 {idx+1}")
        section_text = f"### [{strategy_type}]\n{response}"
        sections.append(section_text)

    join_sections = "\n".join(sections)
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 AI 면접관이다. 다음은 인터뷰 질문, 답변, 평가에 대한 요약이다."),
        ("human", f"""
다음 내용은 각 질문에 대한 요약, 강점, 약점이다:

{join_sections}

이 정보를 바탕으로 종합 피드백을 작성하라.
- 전체적인 강점과 약점
- 향후 개선을 위한 조언
- 직무 적합성 판단 등
""")
    ])

    summary_messages = summary_prompt.format_messages()
    summary_llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    summary_raw = summary_llm.invoke(summary_messages)
    summary_response = StrOutputParser().parse(summary_raw.content)

    print("# 인터뷰 피드백 보고서\n")
    for section in sections:
        print(section)
        print()
    print("### [종합 피드백]")
    print(summary_response)

    return state


# -----------------------------------------------
# 4. Agent 라우팅 로직 및 그래프 정의
# -----------------------------------------------
def route_next(state: InterviewState) -> Literal["generate", "summarize"]:
    return "summarize" if state.get("next_step") == "end" else "generate"

def route_reflection(state: InterviewState) -> Literal["re_evaluate", "decide"]:
    return "re_evaluate" if state.get("next_step") == "re_evaluate_answer" else "decide"

graph_builder = StateGraph(InterviewState)
graph_builder.add_node("evaluate", evaluate_answer)
graph_builder.add_node("reflection", reflection_node)
graph_builder.add_node("re_evaluate", re_evaluation_answer)
graph_builder.add_node("decide", decide_next_step)
graph_builder.add_node("generate", generate_question)
graph_builder.add_node("summarize", summarize_interview)

graph_builder.add_edge(START, "evaluate")
graph_builder.add_edge("evaluate", "reflection")
graph_builder.add_conditional_edges("reflection", route_reflection, {
    "re_evaluate": "re_evaluate",
    "decide": "decide"
})
graph_builder.add_edge("re_evaluate", "decide")
graph_builder.add_conditional_edges("decide", route_next, {
    "generate": "generate",
    "summarize": "summarize"
})
graph_builder.add_edge("generate", END)
graph_builder.add_edge("summarize", END)

graph = graph_builder.compile()


# -----------------------------------------------
# 5. Gradio 인터페이스 정의
# -----------------------------------------------
import gradio as gr
import tempfile

def initialize_state():
    return {
        "state": None,
        "interview_started": False,
        "interview_ended": False,
        "chat_history": []
    }

def upload_and_initialize(file_obj, session_state):
    if file_obj is None:
        return session_state, "파일을 업로드해주세요."

    file_path = file_obj.name
    state = preProcessing_Interview(file_path)
    session_state["state"] = state
    session_state["interview_started"] = True
    first_question = state["current_question"]
    session_state["chat_history"].append(["🤖 AI 면접관", first_question])

    return session_state, session_state["chat_history"]

def chat_interview(user_input, session_state):
    if not session_state["interview_started"]:
        return session_state, "먼저 이력서를 업로드하고 인터뷰를 시작하세요."

    session_state["chat_history"].append(["🙋‍♂️ 지원자", user_input])
    session_state["state"] = update_current_answer(session_state["state"], user_input)
    session_state["state"] = graph.invoke(session_state["state"])

    if session_state["state"]["next_step"] == "end":
        session_state["interview_ended"] = True
        final_summary = "✅ 인터뷰가 종료되었습니다!\n\n"
        for i, turn in enumerate(session_state["state"]["conversation"]):
            final_summary += f"\n**[질문 {i+1}]** {turn['question']}\n**[답변 {i+1}]** {turn['answer']}\n"
            if i < len(session_state["state"]["evaluation"]):
                eval_result = session_state["state"]["evaluation"][i]
                final_summary += f"_평가 - 질문 연관성: {eval_result['질문과의 연관성']}, 답변 구체성: {eval_result['답변의 구체성']}_\n"
        session_state["chat_history"].append(["🤖 AI 면접관", final_summary])
        return session_state, session_state["chat_history"], gr.update(value="")

    else:
        next_question = session_state["state"]["current_question"]
        session_state["chat_history"].append(["🤖 AI 면접관", next_question])
        return session_state, session_state["chat_history"], gr.update(value="")

# Gradio UI 구성
with gr.Blocks() as demo:
    session_state = gr.State(initialize_state())

    gr.Markdown("# 🤖 AI 면접관 \n이력서를 업로드하고 인터뷰를 시작하세요!")

    with gr.Row():
        file_input = gr.File(label="이력서 업로드 (PDF 또는 DOCX)")
        upload_btn = gr.Button("인터뷰 시작")

    chatbot = gr.Chatbot()
    user_input = gr.Textbox(show_label=False, placeholder="답변을 입력하고 Enter를 누르세요.")

    upload_btn.click(upload_and_initialize, inputs=[file_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(chat_interview, inputs=[user_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(lambda: "", None, user_input)

demo.launch(share=True)



Overwriting app.py


## **3. 실행**

In [ ]:
!python app.py

/content/app.py:747: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://793dcdf8b1b3d7b64d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1857: UserWarning: A function (chat_interview) returned too many output values (needed: 2, returned: 3). Ignoring extra values.
    Output components:
        [state, chatbot]
    Output values returned:
        [{'state': {'resume_text': '<이력서> \n홍길동 (Gil-dong Hong) \n이메